In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
%matplotlib inline
import pandas as pd

/Users/bumho/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import json
from pprint import pprint

def getGameJSON(path):
    with open(path) as data_file:    
        data = json.load(data_file)
    return data
        
data = getGameJSON('GSWvsCLE.json')


In [3]:
def getHomeVisitorIds(data):
    visitors = data['events'][0]['visitor']['players']
    home = data['events'][0]['home']['players']

    visitor_ids = []
    home_ids = []


    for visitor in visitors:
        visitor_ids.append(visitor['playerid'])    

    for homie in home:
        home_ids.append(homie['playerid'])  

    return {'visitor': visitor_ids, 'home': home_ids}

getHomeVisitorIds(data)

{'home': [201567,
  2592,
  202681,
  202697,
  2747,
  203521,
  203099,
  202684,
  2760,
  202389,
  2544,
  2210,
  2590],
 'visitor': [201574,
  201575,
  201578,
  2738,
  202691,
  101106,
  2571,
  203546,
  203110,
  201939,
  203105,
  2733,
  203084]}

In [4]:
# read in the court png file
court = plt.imread("fullcourt.png")

headers = ["team_id", "player_id", "x_loc", "y_loc", 
           "radius"]

def playerDFtoList(frame):
    xList = frame['x_loc'].tolist()
    yList = frame['y_loc'].tolist()
    bothList = []
    for i in range(len(xList)):
        bothList.append([ xList[i], yList[i] ])
    return np.array(bothList)

In [5]:
def getMoments(data):
    player_moments = []
    for event in data['events']:
        for moment in event['moments']:
            for player in moment[5]:
                player_moments.append(player)

    df = pd.DataFrame(player_moments, columns=headers)
    return df


In [6]:
def calculateDistance(x1,y1,x2,y2):
     dist = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
     return dist

def distanceAndHandler(moment):
    
    df = getMoments(moment)
    df['DistanceToBall'] = 0.0
    df['DistanceToBasket'] = 0.0
    df['HandlerId'] = 0.0
    handler = ''
    shortest_distance = 100.0
    basket1 = [5.32, 24.8]
    basket2 = [88.55, 24.8]
    halfcourt = 47.0
    
    num_plays = 4
        
    for j in range(0,num_plays):
        index = j + j*10
        
        play = df[index : index + 10]
                
        ballX = df.x_loc[index]
        ballY = df.y_loc[index]

        if ballX > halfcourt:
            dist = calculateDistance(ballX, ballY, basket2[0], basket2[1])
        elif ballX < halfcourt:
            dist = calculateDistance(ballX, ballY, basket1[0], basket1[1])
        
        df.DistanceToBasket[index] = dist
        
        for i in range(index, index+11):
            ballX = df.x_loc[index]
            ballY = df.y_loc[index]
            distToBall = float(calculateDistance(df.x_loc[i], df.y_loc[i], ballX, ballY))
            df.DistanceToBall[i] = float(distToBall)

            if distToBall > 0.0 and distToBall < shortest_distance:
                shortest_distance = distToBall
                handler = df.player_id[i]
    
            df.HandlerId[index] = handler
        
    return df


data = getGameJSON('GSWvsCLE.json')
a = distanceAndHandler(data)

/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bumho/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:117: SettingWithCopyWarning: 
A value is trying to be se

In [7]:
def teamOnOffense(moment):
    
    df = getMoments(moment)

    halfcourtX = 47.0
    if df.x_loc[0] > halfcourtX:
        return df.team_id[6]
    elif df.x_loc[0] < halfcourtX:
        return df.team_id[1]

data = getGameJSON('GSWvsCLE.json')
teamOnOffense(data)

1610612739

In [8]:
def distanceToBasket(moment):
    basket1 = [5.32, 24.8]
    basket2 = [88.55, 24.8]
    halfcourt = 47.0
    
    ballX = moment.x_loc[0]
    ballY = moment.y_loc[0]
    
    if ballX > halfcourt:
        dist = calculateDistance(ballX, ballY, basket2[0], basket2[1])
        return dist
    elif ballX < halfcourt:
        dist = calculateDistance(ballX, ballY, basket1[0], basket1[1])
        return dist    
    
data = getGameJSON('GSWvsCLE.json')
df = getMoments(data)
distanceToBasket(df[0:11])

31.118304104743558

In [19]:
min_5 = 25*60*6*11

def dist(data, ball):
    data["distanceToBall"] = np.sqrt((data["x_loc"] -ball["x_loc"])**2+ (data["y_loc"] -ball["y_loc"])**2)

def hasBall(data):
    data["hasBall"] = data["distanceToBall"].apply(lambda x: 1 if x < 1.8 else 0)
    
def transform(l, ball):
    ball.reset_index(drop=True, inplace =True)    
    result = []
    for player in l:
        player.reset_index(drop=True, inplace =True)
        dist(player, ball)
        hasBall(player)
        result.append(player[(player.hasBall == 1)][["x_loc", "y_loc"]])

    
    return pd.concat(result)

        

In [25]:
# Get moments from the zeroth event
player_moments = []
for event in data['events']:
    for moment in event['moments']:
        for player in moment[5]:
            player_moments.append(player)

df = pd.DataFrame(player_moments, columns=headers)
df = df.head(min_5)

# Get specific player's movements
ball = df[df.player_id==-1]

steph = df[df.player_id==201939]
klay = df[df.player_id==202691]
bogut = df[df.player_id==101106]
green = df[df.player_id==203110]
barnes = df[df.player_id==203084]

lebron = df[df.player_id==2544]
smith = df[df.player_id==2747]
love = df[df.player_id==201567]
irving = df[df.player_id==202681]
mozgov = df[df.player_id==202389]

# convert them to numpy arrays
ballA = playerDFtoList(ball)

stephA = playerDFtoList(steph)
klayA = playerDFtoList(klay)
bogutA = playerDFtoList(bogut)
greenA = playerDFtoList(green)
barnesA = playerDFtoList(barnes)

lebronA = playerDFtoList(lebron)
smithA = playerDFtoList(smith)
loveA = playerDFtoList(love)
irvingA = playerDFtoList(irving)
mozgovA = playerDFtoList(mozgov)

#Get ball radiuses
radii = ball['radius'].tolist()

In [26]:
l = [steph,klay,bogut ,green ,barnes ,lebron,smith,love,irving,mozgov]

a = transform(l, ball)

/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [27]:
a = a.sort_index()
idx = np.unique(a.index, return_index=True)[1]
a= a.iloc[idx]
index = np.arange(0, min_5/11, 1)

a = a.reindex(index, fill_value=0)
a_pos = playerDFtoList(a)

9000

In [32]:
ex = {}
ex["ball"] = ballA.tolist()

ex["steph"] = ballA.tolist()
ex["klay"] = klayA.tolist()
ex["bogut"] = bogutA.tolist()
ex["green"] = greenA.tolist()
ex["barnes"] = barnesA.tolist()


ex["lebron"] = lebronA.tolist()
ex["smith"] = smithA.tolist()
ex["love"] = loveA.tolist()
ex["irving"] = irvingA.tolist()
ex["mozgov"] = mozgovA.tolist()

ex["pos"] = a_pos.tolist()

with open("dump.json", "w") as outfile:
    json.dump(ex, outfile)